In [1]:
import torch

print(torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(device)
#device=torch.device('cpu')########

1.3.1
cuda:0


In [2]:
import os

corpus_name = "mecab_tagged.txt"#"train_model32000.txt"#
position = "./data"
corpus = os.path.join(position, corpus_name)
print(corpus)

./data/mecab_tagged.txt


In [3]:
from Voc import *

In [4]:
printLines(corpus)

['なんと !', 'なんと !\n'] 2 2
['なんと !', '意味 : 都 を 移す こと 、 都 移り\n'] 2 9
['わたし 安定 の たっくん です よー 気づい たら たっくん い ない し 、 いつき ある し 誰 と 交換 し た の か', 'まさかの 増え てる w 羨ましい\n'] 23 5
['まさかの 増え てる w 羨ましい', '連れ が 誰か 引い て た ん です よ 、 きっと それ と 交換 し た ん だ と 思う ん です けど 誰 だっ た の か w あ 、 たっくん は しれっと 買取 に 出し まし た ー\n'] 5 40
['おちつけ よ', 'みんな は げろ\n'] 2 3
['みんな は げろ', '昨日 髪の毛 切っ た ばっかり だ けど この くらい つるっ パゲ に なっ て き ます !\n'] 3 17
['今 は 古 すぎ て 廃盤 の もの が 多い ので 、 いい モンスター が 出る の だ と 高い 値 で 売れ ます w', 'あ 、 やっぱり そう 言う の ある です ね w\n'] 25 10
['あ 、 やっぱり そう 言う の ある です ね w', 'あり ます あり ます w\n'] 10 5
['や だお', '目元 は 知っ てる 、 か あ いい\n'] 2 8
['目元 は 知っ てる 、 か あ いい', '超 過去 写真\n'] 8 3


In [5]:
MIN_COUNT = 3#28#22#
print(corpus_name)
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name)#, datafile, save_dir)
pairs = trimRareWords(voc, pairs, MIN_COUNT)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

mecab_tagged.txt
Start preparing training data ...
Reading lines...
Read 2187212 sentence pairs
Trimmed to 1642304 sentence pairs
Counting words...
Counted words: 255836
keep_words 108420 / 255833 = 0.4238
Trimmed from 1642304 pairs to 1465962, 0.8926 of total

pairs:
['なんと !', 'なんと !']
['なんと !', '意味 : 都 を 移す こと 、 都 移り']
['おちつけ よ', 'みんな は げろ']
['みんな は げろ', '昨日 髪の毛 切っ た ばっかり だ けど この くらい つるっ パゲ に なっ て き ます !']
['あ 、 やっぱり そう 言う の ある です ね w', 'あり ます あり ます w']
['や だお', '目元 は 知っ てる 、 か あ いい']
['目元 は 知っ てる 、 か あ いい', '超 過去 写真']
['いつ でき ます か 。', '今 大丈夫 です !']
['今 大丈夫 です !', 'カフェ の 内部 に 来 て ください']
['絶対 上手いっ て ! 描い て み て', 'それ は ない ね ! ミニー ちゃん ?']


In [6]:
import pickle
# #Load batches for each iteration
# train_pairs,test_pairs = train_test_split(pairs ,test_size=0.2)
# with open('data/train_test_pairs_mecab.pickle', 'wb') as f:
#     pickle.dump((train_pairs,test_pairs), f)
with open("data/train_test_pairs_mecab3min.pickle", "rb") as f:
    #train_test_pairs_32k.pickle
    train_pairs,test_pairs = pickle.load(f)
 
print(voc.name)
print("ちゃんと目的のコーパスファイルを読み込んでいるか確認")
print("train size",len(train_pairs))
print("example",train_pairs[:2])
print("test size",len(test_pairs))
print("example",test_pairs[:2])

mecab_tagged.txt
ちゃんと目的のコーパスファイルを読み込んでいるか確認
train size 1172769
example [['メン 入り の ワカメスープ でし た w', 'もう それ わかめスープ で いい ん じゃ w'], ['私 も 、 あやね ちゃん の スクショ 心待ち に し てる ー', 'も少し 極める から 待っ て すでに 30枚 くらい とり まし た']]
test size 293193
example [['別に 痛く も 痒く も ない けど ね', 'w なんか 、 急 に かっ ちょい い w'], ['おやすみ !', 'ぐんない']]


In [7]:
from model import *

# model.pyのモデルの説明

Get embedding of current input word.
Forward through unidirectional GRU.
Calculate attention weights from the current GRU output from (2).
Multiply attention weights to encoder outputs to get new “weighted sum” context vector.
Concatenate weighted context vector and GRU output using Luong eq. 5.
Predict next word using Luong eq. 6 (without softmax).
Return output and final hidden state.

input_step: one time step (one word) of input sequence batch; shape=(1, batch_size)(1,3)
last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)(1,3,2)
encoder_outputs: encoder model’s output; shape=(max_length, batch_size, hidden_size)(4,3,2)

In [8]:
#訓練するときだけ
print(voc.name)
# Configure models
model_name = "focal_loss_model"
attn_model = 'dot'
# attn_model = 'general'
# attn_model = "concat"
hidden_size = 256
encoder_n_layers = 4
decoder_n_layers = 4
dropout = 0.1
batch_size = 1024
teacher_forcing_ratio = 0.2

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 72000#100000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
                           '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    print("load save file")
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    # checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint["en"]
    decoder_sd = checkpoint["de"]
    encoder_optimizer_sd = checkpoint["en_opt"]
    decoder_optimizer_sd = checkpoint["de_opt"]
    embedding_sd = checkpoint["embedding"]
    voc.__dict__ = checkpoint["voc_dict"]


print("Building encoder and decoder ...")
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = AttnDecoderRNN(
    attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout
)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print("Models built and ready to go!")


mecab_tagged.txt
load save file
Building encoder and decoder ...
Models built and ready to go!


In [9]:
def focalLoss(inp, target, mask, gamma=2, eps=1e-7):
    nTotal = mask.sum()
    logit = torch.gather(inp, 1, target.view(-1, 1)).squeeze(1)
    crossEntropy = -torch.log(logit)# cross entropy
    loss = crossEntropy * (1 - logit) ** gamma # focal loss
    loss = loss.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [10]:
def train(
    input_variable,
    lengths,
    target_variable,
    mask,
    max_target_len,
    encoder,
    decoder,
    embedding,
    encoder_optimizer,
    decoder_optimizer,
    batch_size,
    teacher_forcing_ratio,
    clip,
):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for i in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[: decoder.n_layers]
    
    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_outputs, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = focalLoss(
                decoder_outputs, target_variable[t], mask[t]
            )
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_outputs, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_outputs.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = focalLoss(
                decoder_outputs, target_variable[t], mask[t]
            )
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal


    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    # print(_)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals#, propose_loss.item()*max_target_len
#lossはたぶん1単語あたりのcross entropy lossの値

In [11]:
def test(
    input_variable,
    lengths,
    target_variable,
    mask,
    max_target_len,
    encoder,
    decoder,
    embedding,
    batch_size,
):

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for i in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[: decoder.n_layers]

    
    # Forward batch of sequences through decoder one time step at a time
    

    for t in range(max_target_len):
        decoder_outputs, decoder_hidden = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        # No teacher forcing: next input is decoder's own current output
        _, topi = decoder_outputs.topk(1)
        decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
        decoder_input = decoder_input.to(device)
        # Calculate and accumulate loss
        mask_loss, nTotal = focalLoss(
            decoder_outputs, target_variable[t], mask[t]
        )
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal
    


    return sum(print_losses) / n_totals#, propose_loss.item()*max_target_len



In [12]:
from livelossplot import PlotLosses
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
def trainIters(
    model_name,
    voc,
    train_pairs,
    test_pairs,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    embedding,
    encoder_n_layers,
    decoder_n_layers,
    save_dir,
    n_iteration,
    batch_size,
    teacher_forcing_ratio,
    print_every,
    save_every,
    clip,
    corpus_name,
    loadFilename,
    start_iteration=1,
):

    

    # Initializations
    print("Initializing ...")
    # start_iteration = 1
    print_loss = 0
    print_val_loss = 0
    liveloss = PlotLosses()
    if loadFilename:
        start_iteration = checkpoint["iteration"] + 1
        print('start from',start_iteration,"iteration")
        liveloss = checkpoint["log"]
    

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = batch2TrainData(voc, [random.choice(train_pairs) for _ in range(batch_size)])
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        
        # Ensure dropout layers are in train mode
        encoder.train()
        decoder.train()

        # Run a training iteration with batch
        loss = train(   
            input_variable,
            lengths,
            target_variable,
            mask,
            max_target_len,
            encoder,
            decoder,
            embedding,
            encoder_optimizer,
            decoder_optimizer,
            batch_size,
            teacher_forcing_ratio,
            clip,
        )
        

        print_loss += loss
        
        with torch.no_grad():
            test_batch = batch2TrainData(voc, [random.choice(test_pairs) for _ in range(batch_size)])
            # Extract fields from batch
            input_variable, lengths, target_variable, mask, max_target_len = test_batch
            # Ensure dropout layers are in eval mode
            encoder.eval()
            decoder.eval()
            
            # Run a training iteration with batch
            val_loss = test(     #,val_propose_loss
                input_variable,
                lengths,
                target_variable,
                mask,
                max_target_len,
                encoder,
                decoder,
                embedding,
                batch_size,
            )

            print_val_loss += val_loss
        
        log={}
        log['loss'] = loss
        log['val_loss'] = val_loss
#         log['vector_distance'] = propose_loss
#         log['val_vector_distance'] = val_propose_loss
        liveloss.update(log)
        if iteration % print_every == 0:
            liveloss.draw()

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print_val_loss_avg = print_val_loss / print_every
#             print(
#                 "Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(
#                     iteration, iteration / n_iteration * 100, print_loss_avg
#                 )
#             )
            with open(voc.name[:-4]+'_logs','a') as f:
                f.write("\nIteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}; Average validation loss: {:.4f}".format(
                    iteration, iteration / n_iteration * 100, print_loss_avg, print_val_loss_avg
                )
                       )
            
            print_loss = 0
            print_val_loss = 0
            
            graph_x = range(len(liveloss.logs))
            pdf = PdfPages(voc.name[:-4]+'_logs.pdf')
            plt.figure()
            plt.plot(graph_x, [i['loss'] for i in liveloss.logs])
            plt.plot(graph_x, [i['val_loss'] for i in liveloss.logs])
            plt.legend(('training','validation'))
            pdf.savefig()
            pdf.close()
            

        # Save checkpoint
        if iteration % save_every == 0:
            directory = os.path.join(
                save_dir,
                model_name,
                corpus_name,
                "{}-{}_{}_{}".format(encoder_n_layers, decoder_n_layers, hidden_size, teacher_forcing_ratio),
            )

            if not os.path.exists(directory):
                os.makedirs(directory)

            torch.save(
                {
                    "iteration": iteration,
                    "en": encoder.state_dict(),
                    "de": decoder.state_dict(),
                    "en_opt": encoder_optimizer.state_dict(),
                    "de_opt": decoder_optimizer.state_dict(),
                    "log": liveloss,
                    "voc_dict": voc.__dict__,
                    "embedding": embedding.state_dict(),
                },
                os.path.join(directory, "{}_{}.tar".format(iteration, "checkpoint")),
            )




In [13]:
# パラメータカウント関数
print("パラメータ数をカウント")
print("encoderのパラメータ数")
parameters_count(encoder)
print("decoderパラメータ数")
parameters_count(decoder)
#print(encoder)

パラメータ数をカウント
encoderのパラメータ数
32093952
decoderパラメータ数
57331335


In [14]:
import torch.optim as optim

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.2####
learning_rate = 0.0001#学習率を0.001とか大きめにとると（大きく無いと思うけど）途中でnanを吐き始める
decoder_learning_ratio = 2.0
n_iteration = 500000
print_every = 500
save_every = 1000
#cal_method='cos'
print('the number of data is',len(pairs))


# Initialize optimizers
print("Building optimizers ...")
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(
    decoder.parameters(), lr=learning_rate * decoder_learning_ratio
)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("count parameters")

print("Starting Training!")
trainIters(
    model_name,
    voc,
    train_pairs,
    test_pairs,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    embedding,
    encoder_n_layers,
    decoder_n_layers,
    save_dir,
    n_iteration,
    batch_size,
    teacher_forcing_ratio,
    print_every,
    save_every,
    clip,
    corpus_name,
    loadFilename,
)

the number of data is 1465962
Building optimizers ...
count parameters
Starting Training!
Initializing ...
start from 72001 iteration
Training...


RuntimeError: CUDA out of memory. Tried to allocate 424.00 MiB (GPU 0; 10.73 GiB total capacity; 9.69 GiB already allocated; 97.19 MiB free; 70.41 MiB cached)

In [ ]:
from matplotlib import pyplot as plt

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    logit = torch.gather(inp, 1, target.view(-1, 1))
    crossEntropy = -torch.log(logit.squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

def focalLoss(inp, target, mask, gamma=1, eps=1e-7):
    nTotal = mask.sum()
    logit = torch.gather(inp, 1, target.view(-1, 1)).squeeze(1)
    crossEntropy = -torch.log(logit)# cross entropy
    loss = crossEntropy * (1 - logit) ** gamma # focal loss
    loss = loss.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()


xitem = [i/100 for i in range(1,101)]
for ite in [ 0.5, 1, 2, 5]:
    focalloss = []
    normalloss = []
    for x in range(1,101):
        sample = [[(1-x/100)/3, x/100, (1-x/100)/3, (1-x/100)/3]]
        decoder_output = torch.tensor(sample)
        mask = torch.BoolTensor([[1]])
        target_variable = torch.LongTensor([[1]])
        normalloss.append(maskNLLLoss(decoder_output, target_variable[0], mask[0])[0].item())
        focalloss.append(focalLoss(decoder_output, target_variable[0], mask[0],gamma=ite)[0].item())
        
    plt.plot(xitem,focalloss,label=('focal gamma='+str(ite)))
    
plt.plot(xitem,normalloss,label=('normal'))
plt.legend()
plt.show()